<center><h1><strong> Informe Tarea 1 - Beer Clustering </strong></h1></center>

<center><h3> Matías Pacheco &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;201204619-k</h3></center>
<center><h3> Juan Pablo Castillo Vera &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 201204595-9</h3></center>

## Descripción del dataset

El dataset Beer Reviews es una colección de 1.590.892 críticas de cervezas que se encuentran disponibles en el sitio http://beeradvocate.com/. Éstas críticas toman en consideración características como aroma, apariencia y sabor para asignarle un puntaje a la cerveza que está siendo evaluada.

Cada registro del dataset completo está compuesto por las siguientes columnas:

|Columna             |Significado            |Tipo       |
|---                 |---                    |---        |
|brewery_id          |ID cervecería          |Numérico   |
|brewery_name        |Nombre Cervecería      |Texto      |
|review_time         |Tiempo de evaluación   |Numérico   |
|review_overall      |Puntaje del review     |Numérico   |
|review_aroma        |Puntaje aroma          |Numérico   |
|review_appearence   |Puntaje apariencia     |Numérico   |
|review_profilename  |Nombre del evaluador   |Texto      |
|beer_style          |Tipo de cerveza        |Texto      |
|review_palate       |Puntaje paladar        |Numérico   |
|review_taste        |Puntaje gusto          |Numérico   |
|beer_abv            |Grado alcohólico       |Numérico   |
|beer_beerid         |ID cerveza             |Numérico   |

Cabe señalar que como pre-procesamiento de los datos, se setearon en cero todos aquellos campos numéricos del dataset que estuvieran vacíos (string '').


## Análisis de Clustering

Se realizará un análisis de clustering con el dataset con cada uno de los siguientes cinco algoritmos:
* K-Means
* Minibatch K-Means
* HAC Complete
* Ward
* DBScan

### Observaciones y Supuestos

* Antes del procesamiento de los datos, éstos fueron normalizados BLABLABLABLBALBALBALBALBALBALBALBLBA.. JP DO YOUR THING!!!!

In [ ]:
from sklearn.preprocessing import StandardScaler
...
reviews = StandardScaler().fit_transform(reviews)

* Para reducir la dimensionalidad de los vectores, y posteriormente poder graficar los resultados en un minifold 2D, se utilizó Principal Component Analysis (PCA).

In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

dataset = pd.read_csv('beer_reviews.csv')
...
reviews = PCA(n_components = 2).fit_transform(reviews)

* Se tomaron los campos **brewery_id** y **beer_beerid** como las formas numéricas de las etiquetas **brewery_name** y **beer_type**. De esta forma, **brewery_id** es equivalente a **brewery_name**, y **beer_beerid** es equivalente a **beer_type**. 

* Todos los análisis que se muestran a continuación se realizaron entrenando los algoritmos de clustering con las siguientes 6 variables enteras del dataset: review_overall, review_aroma, review_appearance, review_palate, review_taste, beer_abv. Las demás variables no numéricas fueron descartadas.

In [ ]:
reviews = pd.DataFrame({'review_overall':dataset['review_overall'], 'review_aroma':dataset['review_aroma'], 'review_appearance':dataset['review_appearance'], 'review_palate':dataset['review_palate'], 'review_taste':dataset['review_taste'],'beer_abv':dataset['beer_abv']})

* Dado que los a los algortimos no se les pasa como parte del dataset las etiquetas que se quieren evaluar, éste proceso de clustering es no supervisado. Es por esto que posterior a la ejecución de los algoritmos de clustering se evalúa la calidad de los clusters encontrados utilizando la función Normalized Mutual Information (NMI), a la cual se le pasa como parámetro las etiquetas reales y las etiquetas obtenidas por los algoritmos, retornando un valor entre 0 y 1 que determina la calidad de los clusters.

In [ ]:
from sklearn.metrics import normalized_mutual_info_score as NMI
...
labels = list(clustering_algorithm.labels_)
NMI(real_labels, labels)

* En este informe se mostrarán sólo los gráficos resultantes del análisis del dataset con cada algoritmo. El código fuente está disponible en https://github.com/matipacheco/Tarea1-MD.

### K-Means

Los parámetros de K-Means utilizados para realizar el análisis fueron los siguientes:

* **init     = 'k-means++'**. Para una convergencia más rápida.
* **n_init   = 10**. Se comprobó que incrementar la cantidad de veces que se ejecuta el algoritmo no mejora significativamente el resultado, sólo se produce un pequeño cambio en la posición de los centroides.
* **max_iter = 300 (default)**. Se comprobó que incrementar la cantidad máxima de iteraciones del algoritmo no mejora el resultado.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

Es importante destacar que se puede determinar la cantidad óptima de clusters verificando el valor obtenido por la evaluación NMI. Índices más grandes indican mejores valores de k. 

Dicho lo anterior, se ejecutó el algoritmo K-Means múltiples veces, probando con distintos valores para el parámetro k, con valores desde k = 2 hasta k = 100 clusters. Para la etiqueta **beer_beerid** con **k = 98** se obtiene el mejor índice NMI, NMI = 0.3003. Del mismo modo, para la etiqueta **brewery_id** también se tiene una cantidad óptima de clusters **k = 98**, con un índice NMI =  0.1299.

A continuación se muestra la imagen que resulta de la ejecución K-Means con 98 clusters. 

<center><img src="PlotsKMeans/kmeans.png"></center>
<center><small>Figura 1: Clustering utilizando K-Means.</small></center>


### Minibatch K-Means

Los parámetros de Minibatch K-Means utilizados para realizar el análisis fueron los siguientes:

* **n_init   = 100**. Se comprobó que incrementar la cantidad de veces que se ejecuta el algoritmo produce mejoras en el posicionamiento de los centroides.
* **batch_size = 1000**. Se comprobó que incrementar el tamaño de los batches, mejora el posicionamiento de los centroides de los clusters, pero para valores mayores a 1000 este cambio es muy poco significativo.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

Al igual que para K-Means, se ejecutó Minibatch K-Means con k = 2 hasta k = 100. En este caso, para la etiqueta **beer_beerid** se tiene una cantidad óptima de clusters **k = 100**, con un índice NMI = 0.2998. Mientras que para la etiqueta **brewery_id** se tiene un índice NMI máximo igual a 0.1299, utilizando la misma cantidad de clusters.

A continuación se muestra la imagen que resulta de la ejecución Minibatch K-Means con 100 clusters. 

<center><img src="PlotsMinibatchKMeans/minibatch.png"></center>
<center><small>Figura 2: Clustering utilizando Minibatch K-Means.</small></center>


### HAC Complete

En primer lugar, es necesario aclarar que para realizar los análisis con el algoritmo HAC, se utilizó un subconjunto de 20.000 elementos del dataset, a diferencia de los dos algoritmos anteriores en los que se analizó el dataset completo. Esto se debe a que para un número mayor de elementos la ejecución del algoritmo consume muchísimos recursos del computador, y por falta de memoria y de componentes hardware más poderosas, el algoritmo siempre terminaba cayéndose.

Los parámetros de HAC utilizados para realizar el análisis fueron los siguientes:

* **linkage   = 'complete'**. Por restricción de la tarea.
* **affinity = 'euclidean'**. Se comprobó que sólo con la métrica cosine se obtenían clusters distintos. Las métricas euclidean, manhattan, l1 y l2 formaban los mismos clusters.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos.

Al igual que los dos algoritmos anteriores, se ejecutó HAC pasándole como parámetro valores de k = 2 hasta k = 100. En este caso, para la etiqueta **beer_beerid** se tiene una cantidad óptima de clusters **k = 100**, con un índice NMI = 0.4227. Mientras que para la etiqueta **brewery_id** se tiene un índice NMI máximo igual a 0.2332, para **k = 98** clusters. Se puede apreciar claramente que este algoritmo retorna valores de NMI mucho mejores que K-Means y Minibatch K-Means, pero no se puede asegurar que efectivamente HAC sea mejor que los otros dos, dado que el algoritmo se probó con un subset correspondiente al 0.01% del total de datos.

Sin embargo, hay que destacar que para este algoritmo, y a diferencia de los dos anteriores, para cada atiqueta se obtuvieron valores distintos de k. Dicho eso, a continuación se muestra la imagen que resulta de la ejecución de  HAC con 98 y con 100 clusters.

<center><img src="PlotsHAC/hac98.png"></center>
<center><small>Figura 3: Clustering utilizando HAC con 98 clusters.</small></center>

<center><img src="PlotsHAC/hac100.png"></center>
<center><small>Figura 4: Clustering utilizando HAC con 100 clusters.</small></center>


### Ward

Antes de comenzar el analisis, es necesario aclarar que el algoritmo Ward solo se pudo utilizar un subconjunto de 9.000 elementos del dataset. Esto se debe a que para un número mayor de elementos la ejecución del algoritmo consume muchísimos recursos del computador, y por falta de hardware y memoria el algoritmo siempre terminaba cayéndose. Por lo que para obtener resultados, se utiliza una muestra poco significativa (9.000 de 1.590.892 datos).

A continuación se muestran los resultados del clustering por *brewery_id* para 2, 3, 4 y 5 clusters:

<center><img src="PlotsWard/ward1.png"></center>
<center><small>Figura 8: Clustering por brewery_id utilizando Ward para 2, 3, 4 y 5 clusters.</small></center>

Como puede verse, la separacion entre los clusters es poco notoria, pero comparando los 4 resultados, con k = 4 se nota la mejor separacion de los datos.

A continuación se muestran los resultados del clustering por *beer_beerid* para 2, 3, 4 y 5 clusters:

<center><img src="PlotsWard/ward2.png"></center>
<center><small>Figura 9: Clustering por beer_beerid utilizando Ward para 2, 3, 4 y 5 clusters.</small></center>

Al igual que con la otra etiqueta, la separacion entre clusters sigue siendo poco notoria. Pero con k = 3 se nota la mejor separacion de los datos, por lo tanto, con la etiqueta brewery_id existen 4 clusters y con la etiqueta beer_beerid existen 3 clusters.

### DBScan


Para DBScan, solo se logro utilizar un subconjunto de 150.000 elementos del dataset por problemas de memoria y hardware similares que cuando se utilizo HAC.

Los parámetros de DBScan utilizados para realizar el análisis fueron los siguientes:

* **eps   = 0.5**. Para tener un radio maximo bueno y facilitar la generacion de core points.
* **min_samples = 30**. Se comprobo experimentando que este es un valor estable para alcanzar un buen numero de clusters.

Los demás parámetros se dejaron en su valor por default, ya que la modificación de éstos no produjeron cambios significativos en los resultados obtenidos. A continuacion se mostraran los graficos haciendo diferencia por color de los clusters y por tamaño (circulos mas grandes son *Core Points* y los puntos negros son *Noisy Points*). 

El resultado para ambas etiquetas fueron los siguientes:

<center><img src="PlotsDBScan/dbscan_etiqueta1_1.png"></center>
<center><small>Figura 10: Clustering por brewery_id utilizando DBScan.</small></center>

Como puede notarse, hay bastante Noisy Points (puntos negros), lo que se considera como los datos que no quedaron en ninguno de los 239 clusters. Para la otra etiqueta tambien hay muchos Noise Points:

<center><img src="PlotsDBScan/dbscan_etiqueta2_1.png"></center>
<center><small>Figura 11: Clustering por beer_beerid utilizando DBScan.</small></center>

Como se puede ver, aunque se haya tomado menos del 10% de los datos, el algoritmo DBScan da buenos resultados, pero estos no se pueden comparar a los algoritmos anteriores ya que la cantidad de clusters es significativamente alta para ambas etiquetas. El problema es que DBScan es demasiado sensible al eps y min_samples, por lo tanto al aumentar el eps a un valor grande, la cantidad de clusters se reduce significativamente como en el siguiente ejemplo:

<center><img src="PlotsDBScan/dbscan_etiqueta2_2.png"></center>
<center><small>Figura 12: Clustering por beer_beerid utilizando DBScan con eps = 50.</small></center>

Al comparar lla figura 11 y 12 con etiqueta por *brewery_id*, al aumentar en magnitud 100 veces el eps, se logra reducir de 649 clusters a 271, esto demuestra la sensibilidad del algoritmo. Por otro lado los Noisy points se redujeron bastante respecto a la figura 11.

Ahora, si el eps se disminuye un poco, tambien se nota la sensibilidad, como en la siguiente figura:

<center><img src="PlotsDBScan/dbscan_etiqueta1_2.png"></center>
<center><small>Figura 13: Clustering por brewery_id utilizando DBScan con eps = 0.3.</small></center>

Al comparar la figura 10 y 13 se puede notar que al modificar el eps de 0.5 a 0.3, se aumento de 239 a 606, demostrando nuevamente que el algoritmo es demasiado sensible al radio maximo dado por el eps.




## Conclusiones

¿Qué atributo, Nombre de cervecería o Tipo de cerveza, describe mejor a los cluster como etiquetas
de clase, según los resultados obtenidos previamente? ¿Hay mejores marcas que otras en
relación a las evaluaciones obtenidas? ¿Se puede definir algún criterio para determinar el mejor
tipo de cerveza? Comente